- https://making.lyst.com/lightfm/docs/lightfm.html
- modify mistake in valid label
- consider item, user features
- change weight and range *
- add more features

In [1]:
import os
import tqdm

import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from sklearn import preprocessing

import itertools
from multiprocessing import Pool

SEED = 42
np.random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

VALID = True

In [2]:
def apk(actual, predicted, k=12):
    if len(predicted)>k:
        predicted = predicted[:k]
        
    score = 0.0
    num_hits = 0.0
    
    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=12):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

## Configs

In [3]:
# default number of recommendations
K = 12
EPOCHS = 100
LEARNING_RATE = 0.1 
NO_COMPONENTS = 50
NO_THREADS = 32
ITEM_ALPHA=1e-6
USER_ALPHA=1e-6

num_cores=4

## Load the data

In [4]:
main_dir = "../input/h-and-m-personalized-fashion-recommendations"
images_dir = main_dir+"/images/" 
customers = pd.read_csv(main_dir+"/customers.csv")
articles = pd.read_csv(main_dir+"/articles.csv", dtype={'article_id': str})
sample_submission = pd.read_csv(main_dir+"/sample_submission.csv", dtype={'article_id': str})
train = pd.read_csv(main_dir+'/transactions_train.csv',  dtype={'article_id': str}, parse_dates=['t_dat'])

# features

In [5]:
train["t_dat"] = pd.to_datetime(train["t_dat"])

date_week_df = train.drop_duplicates("t_dat")[["t_dat"]].reset_index(drop=True)
date_week_df['week_no'] = (date_week_df['t_dat'] + pd.DateOffset(days=5)).dt.week
date_week_df["week_no"] = date_week_df["week_no"].diff(1)
date_week_df["week_no"].fillna(0, inplace=True)
date_week_df["week_no"] = date_week_df["week_no"] != 0
date_week_df["week_no"] = date_week_df["week_no"].cumsum()

train = pd.merge(train, date_week_df, on="t_dat", how="left")
test_week = train.week_no.max()

del date_week_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  after removing the cwd from sys.path.


In [6]:
if VALID:
    tmp = train[train.week_no < test_week].reset_index(drop=True).copy()
else:
    tmp = train.copy()

# count encoding
tmp = pd.merge(tmp, articles[["article_id", "index_group_name"]], on='article_id', how='left')
mp = {'Ladieswear':1, 'Baby/Children':0.5, 'Menswear':0, 'Sport':0.5, 'Divided':0.5} 
tmp["gender"] = tmp["index_group_name"].map(mp)
user_g = tmp.groupby('customer_id').gender.mean().reset_index()
    
item_count = tmp["article_id"].value_counts().reset_index()
item_count.columns = ["article_id", "item_count"]
user_count = tmp["customer_id"].value_counts().reset_index()
user_count.columns = ["customer_id", "user_count"]

tmp1 = tmp[tmp.sales_channel_id==1]["article_id"].value_counts().reset_index()
tmp1.columns = ["article_id", "count_in1"]

tmp2 = tmp[tmp.sales_channel_id==2]["article_id"].value_counts().reset_index()
tmp2.columns = ["article_id", "count_in2"]

sales_channel_count = pd.merge(tmp1, tmp2, on="article_id", how="outer").fillna(0)
sales_channel_count["channel_ratio"] = sales_channel_count["count_in1"] / (sales_channel_count["count_in2"]+0.1)

user_last_week = tmp.groupby("customer_id")["week_no"].max().reset_index()
user_last_week.columns = ["customer_id", "last_week"]

user_num_week = tmp.groupby("customer_id")["week_no"].nunique().reset_index()
user_num_week.columns = ["customer_id", "week_num"]

del tmp1, tmp2, tmp

customers = pd.merge(customers, user_count, on = "customer_id", how="left")
customers["user_count"].fillna(0, inplace=True)
customers = pd.merge(customers, user_last_week, on = "customer_id", how="left")
customers["last_week"].fillna(-1, inplace=True)
customers = pd.merge(customers, user_num_week, on = "customer_id", how="left")
customers["week_num"].fillna(-1, inplace=True)
customers = pd.merge(customers, user_g, on = "customer_id", how="left")
customers["gender"].fillna(0.75, inplace=True)
del user_count, user_last_week, user_num_week, user_g

articles = pd.merge(articles, item_count, on = "article_id", how="left")
articles = pd.merge(articles, sales_channel_count, on = "article_id", how="left")
articles["item_count"].fillna(0, inplace=True)
articles.fillna(-1, inplace=True)
del item_count, sales_channel_count

In [7]:
# item
article_columns = [i for i in articles.columns if "int" in str(articles[i].dtype)]
article_columns += ["item_count", "count_in1", "count_in2", "channel_ratio"]
articles = articles[['article_id'] + article_columns]

article_columns

['product_code',
 'product_type_no',
 'graphical_appearance_no',
 'colour_group_code',
 'perceived_colour_value_id',
 'perceived_colour_master_id',
 'department_no',
 'index_group_no',
 'section_no',
 'garment_group_no',
 'item_count',
 'count_in1',
 'count_in2',
 'channel_ratio']

In [8]:
# user
user_columns = ['FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'postal_code']
customers.fillna(-1, inplace=True)
for feat in user_columns:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(customers[feat]))
    customers[feat] = lbl.transform(list(customers[feat]))
user_columns += ['age', "user_count", "last_week", "week_num", "gender"]
user_columns

['FN',
 'Active',
 'club_member_status',
 'fashion_news_frequency',
 'postal_code',
 'age',
 'user_count',
 'last_week',
 'week_num',
 'gender']

# dataset

In [9]:
dataset = Dataset()
dataset.fit(users=customers['customer_id'], 
            items=articles['article_id'],
            user_features = user_columns,
            item_features = article_columns) 

num_users, num_topics = dataset.interactions_shape()
print(f'Number of users: {num_users}, Number of topics: {num_topics}.')

#Get the mappings
uid_map, ufeature_map, iid_map, ifeature_map = dataset.mapping() 
inv_uid_map = {v:k for k, v in uid_map.items()}
inv_iid_map = {v:k for k, v in iid_map.items()}

Number of users: 1371980, Number of topics: 105542.


In [10]:
item_last_week = train.groupby("article_id")["week_no"].max().reset_index()
weak_weight_items = item_last_week[item_last_week.week_no > 100]["article_id"].values #80
weak_weight_id = [iid_map[ele] for ele in weak_weight_items]
del item_last_week

In [11]:
if VALID:
    train_set = train[train.week_no < test_week].copy()
    train_ser = train_set.drop_duplicates(["customer_id", "article_id"]).reset_index(drop=True)
    (interactions, weights) = dataset.build_interactions(train_set.iloc[:, 1:3].values)

    valid = train[train.week_no == test_week].groupby('customer_id')['article_id'].apply(list).reset_index()
    valid = valid.rename({'article_id':'prediction'},axis=1)
    valid['prediction'] = valid.prediction.apply(lambda x: ' '.join([str(k) for k in x]))

    inactive_users = list(set(train_set.customer_id) - set(train_set[train_set.week_no > test_week - 10]["customer_id"]))
    cold_users = list(set(valid.customer_id) - set(train_set.customer_id))
    del train_set
else:
    (interactions, weights) = dataset.build_interactions(train.iloc[:, 1:3].values)
    
dataset.build_item_features(((articles.iloc[index]["article_id"], article_columns) for index in range(articles.shape[0])))
dataset.build_user_features(((customers.iloc[index]["customer_id"], user_columns) for index in range(customers.shape[0])))

print(interactions.shape)
del train

(1371980, 105542)


In [12]:
change_index = np.isin(weights.col, weak_weight_id)
weights.data[change_index] = 3.0 #2.0

# fitting

In [13]:
model = LightFM(loss='bpr', 
                no_components=NO_COMPONENTS, 
                learning_rate=LEARNING_RATE,                 
                random_state=np.random.RandomState(SEED),
                user_alpha=USER_ALPHA,
                item_alpha=ITEM_ALPHA
               )

model.fit(interactions=interactions, 
          epochs=EPOCHS, 
          verbose=1, 
          num_threads=NO_THREADS,
          sample_weight=weights,
         )

Epoch: 100%|██████████| 100/100 [1:19:35<00:00, 47.76s/it]


# Submission

In [14]:
if VALID:
    test_X = valid.customer_id.values
else:
    test_X = sample_submission.customer_id.values    

lfn_user = lambda x: uid_map[x]
test_X_m = [lfn_user(tx) for tx in test_X]

print(len(test_X_m))

68984


In [15]:
def predict_submission():
    df_split = np.array_split(test_X_m, num_cores)
    chunk_size = len(df_split[0])
    
    chunk1 = list(df_split[0])
    chunk2 = list(df_split[1])
    chunk3 = list(df_split[2])
    chunk4 = list(df_split[3])
    
    #list of items for multiprocessing, 4 since using 4 cores
    all_chunks = [chunk1, chunk2, chunk3, chunk4]
    
    pool = Pool(num_cores)
    result = pool.map(make_predict, all_chunks)
    
    pred_combined = list(itertools.chain(result[0], result[1], result[2], result[3]))

    return pred_combined

def make_predict(usr_chunk):
    customer_ids = []
    preds = []
    item_array = np.array(list(iid_map.values()))

    for usr_ in usr_chunk:
        m_opt = model.predict(np.array([usr_] * len(iid_map)), item_array)
        pred = np.argsort(-m_opt)[:K]
        preds.append(' '.join([inv_iid_map[p] for p in pred]).strip())
    
    preds = np.array(preds).reshape(-1, 1)
    return preds

In [16]:
%%time
final_predictions = predict_submission()

CPU times: user 1.11 s, sys: 543 ms, total: 1.65 s
Wall time: 11min 57s


In [17]:
final_sub = pd.DataFrame(data=np.concatenate((np.array(test_X).reshape(-1,1), np.array(final_predictions)), axis=1).reshape(-1, 2),
                         columns=['customer_id', 'prediction'])

sample_submission.loc[sample_submission.customer_id.isin(final_sub.customer_id), 'prediction'] = ( 
    final_sub['prediction']
    )
sample_submission.to_csv('submission.csv', index=False)

In [18]:
# local score
if VALID:
    sub_check = final_sub.copy()
    sub_check = sub_check.set_index('customer_id').reset_index()
    print("t", mapk(valid.prediction.str.split(), sub_check.prediction.str.split(), k=12))    
    print("i", 
            mapk(valid[valid.customer_id.isin(inactive_users)].prediction.str.split(), 
               sub_check[sub_check.customer_id.isin(inactive_users)].prediction.str.split(), k=12),
            valid[valid.customer_id.isin(inactive_users)].shape[0])
    print("c", 
            mapk(valid[valid.customer_id.isin(cold_users)].prediction.str.split(), 
               sub_check[sub_check.customer_id.isin(cold_users)].prediction.str.split(), k=12),
            valid[valid.customer_id.isin(cold_users)].shape[0])
    
    print("  ")
    print(sub_check["prediction"].value_counts())  

t 0.006192260190338804
i 0.006540853482227439 19048
c 6.543132328308207e-05 5572
  
0428291001 0376172001 0649551007 0754362003 0500435039 0233091021 0676111001 0725718001 0126589010 0562211012 0613666005 0780418001    5572
0120129001 0534164001 0215589001 0300024058 0567532004 0775382001 0752657001 0433444001 0510465001 0442915001 0262277011 0469562002       4
0120129001 0534164001 0215589001 0567532004 0300024058 0775382001 0752657001 0433444001 0510465001 0442915001 0469562002 0262277011       3
0120129001 0534164001 0215589001 0300024058 0567532004 0775382001 0433444001 0752657001 0510465001 0262277011 0469562002 0442915001       2
0573937001 0294008002 0652924004 0751551001 0750330002 0640542002 0652924010 0624257001 0797892001 0294008005 0750330003 0698387001       2
                                                                                                                                       ... 
0832453005 0832453002 0832473001 0832453001 0832473003 0879944002 0907696001